In [1]:
#imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import boto3
import os
import time
from time import sleep
import json
from datetime import datetime
import pandas as pd
!pip install wordcloud
from wordcloud import WordCloud, STOPWORDS 
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')
import random

     |████████████████████████████████| 366 kB 24.6 MB/s            


In [2]:
bucketname = "recommendation-storage"
my_region = boto3.session.Session().region_name
print(my_region)

s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-1':
        s3.create_bucket(Bucket=bucketname)
    print("S3 bucket created sucessfully")
except Exception as e:
    print("S3 error: " + str(e))
    
prefix = 'kmeans-clustering'
output_path = 's3://{}/{}/output'.format(bucketname,prefix)
print(output_path)

us-east-1
S3 bucket created sucessfully
s3://recommendation-storage/kmeans-clustering/output


In [7]:
data_key = 'ratings.csv'
data_location = 's3://{}/{}'.format(bucketname, data_key)
title_ratings = pd.read_csv(data_location,sep=',')

data_key = 'movies.csv'
data_location = 's3://{}/{}'.format(bucketname, data_key)
title_movies = pd.read_csv(data_location,sep=',')
data_key = 'links.csv'
data_location = 's3://{}/{}'.format(bucketname, data_key)
title_links = pd.read_csv(data_location,sep=',')
data_key = 'tags.csv'
data_location = 's3://{}/{}'.format(bucketname, data_key)
title_tags = pd.read_csv(data_location,sep=',')


model_data = pd.merge(title_ratings,title_movies, on = 'movieId')


model_data.to_csv('merged_data.csv')

boto3.Session().resource('s3').Bucket(bucketname).Object(os.path.join(prefix, 'merged_data.csv')).upload_file('merged_data.csv')


In [8]:
data_dir = "data"
!mkdir $data_dir

!cd $data_dir && wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!cd $data_dir && unzip ml-latest-small.zip
dataset_dir = data_dir + "/ml-latest-small/"
!ls $dataset_dir

--2022-04-19 20:51:57--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  4.75MB/s    in 0.2s    

2022-04-19 20:51:58 (4.75 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  
links.csv  movies.csv  ratings.csv  README.txt	tags.csv


In [15]:
project_data = pd.read_csv(dataset_dir + '/result_final_1 (1).csv')
print(project_data.info())
project_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98386 entries, 0 to 98385
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   movieId    98386 non-null  int64  
 1   imdbId     98386 non-null  int64  
 2   tmdbId     98386 non-null  int64  
 3   title      98386 non-null  object 
 4   genres     98386 non-null  object 
 5   rating     98386 non-null  float64
 6   userId     98386 non-null  int64  
 7   tag        98386 non-null  object 
 8   timestamp  98386 non-null  int64  
 9   userId.1   98386 non-null  int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 7.5+ MB
None


,movieId,imdbId,tmdbId,title,genres,rating,userId,tag,timestamp,userId.1
0,1,114709,862,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,1,pixar,1139045764,336
1,1,114709,862,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,1,pixar,1137206825,474
2,1,114709,862,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,1,fun,1525286013,567
3,3,113228,15602,Grumpier Old Men (1995),Comedy|Romance,4.0,1,moldy,1143424860,289
4,3,113228,15602,Grumpier Old Men (1995),Comedy|Romance,4.0,1,old,1143424860,289


In [22]:
watched_df = project_data.copy()
watched_df = watched_df[watched_df['rating'] > 3]
watched_df = watched_df[['userId', 'movieId', 'timestamp','genres','title']]
watched_df['EVENT_TYPE']='watch'

clicked_df = project_data.copy()
clicked_df = clicked_df[clicked_df['rating'] > 1]
clicked_df = clicked_df[['userId', 'movieId', 'timestamp', 'genres','title']]
clicked_df['EVENT_TYPE']='click'

interactions_df = clicked_df.copy()
interactions_df = interactions_df.append(watched_df)
interactions_df.sort_values("timestamp", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')

In [25]:
interactions_df.rename(columns = {'userId':'USER_ID', 'movieId':'ITEM_ID', 
                              'timestamp':'TIMESTAMP','genres':'GENRES'}, inplace = True)
print(interactions_df.head())
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

       USER_ID  ITEM_ID   TIMESTAMP         GENRES              title  \
43159      262       28  1137180037  Drama|Romance  Persuasion (1995)   
43159      262       28  1137180037  Drama|Romance  Persuasion (1995)   
5658        33       28  1137180037  Drama|Romance  Persuasion (1995)   
5658        33       28  1137180037  Drama|Romance  Persuasion (1995)   
78591      474       28  1137180037  Drama|Romance  Persuasion (1995)   

      EVENT_TYPE  
43159      click  
43159      watch  
5658       watch  
5658       click  
78591      click  
